In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import random
from time import sleep

import spotipy
import sys
from spotipy.oauth2 import SpotifyClientCredentials

In [3]:
    url = "https://www.billboard.com/charts/hot-100"
    response = requests.get(url)
    soup = BeautifulSoup(response.content,"html.parser")

    song_name = []

    for song in soup.select("span.chart-element__information__song.text--truncate.color--primary"):
        song_name.append(song.get_text())

    artist_name = []

    for artist in soup.select("span.chart-element__information__artist.text--truncate.color--secondary"):
        artist_name.append(artist.get_text())

    billboard = pd.DataFrame({
        "song_name":song_name,
        "artist":artist_name,})

    billboard["song_name"] = billboard["song_name"].apply(lambda x: x.lower())
    billboard["artist"] = billboard["artist"].apply(lambda x: x.lower())
    
# Asking for a song

    spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="ff84e02be03141a2b8bf064b47dce0ba",client_secret="2d8b7ea03f734d87a4d21689e8a4ce94"))

    song_not_top_100 = []
    artist_not_top_100 = []

    hot_song = random.choice(billboard["song_name"])

    user_input = input("Name of the song you like. If its in the top 100 I'll recommend you a new one?: ").lower()

    if user_input in list(billboard["song_name"]):
        sleep(1)
        print("We recommend you hear: ", hot_song)

    elif user_input not in list(billboard["song_name"]): 
        song_not_top_100.append(user_input)
        sleep(1)
        print("Song not in top 100")   
        artist_input = input("Who is the artist: ")
        artist_not_top_100.append(artist_input)
        search_name = " ".join(song_not_top_100 + artist_not_top_100)

    results = spotify.search(q=(search_name), limit = 1)
    uri_song_chosen = results["tracks"]["items"][0]["uri"]    

    # Playlist & Clustering


    table_of_tables = pd.read_csv("tablesaved").drop(columns="Unnamed: 0")  #function used for table in the end

    from sklearn import datasets
    from sklearn.preprocessing import StandardScaler
    from sklearn.cluster import KMeans

    cluster_table = table_of_tables[["danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness","liveness","valence","duration_ms"]]
    scaler = StandardScaler()
    table_of_tables_scaled = scaler.fit_transform(cluster_table)

    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters=7,random_state=1234)
    kmeans.fit(table_of_tables_scaled)

    clusters = kmeans.predict(table_of_tables_scaled)
    table_of_tables["clusters"] = clusters

    # Music Selector

    audio_features = spotify.audio_features(uri_song_chosen)
    user_audio_feature_dataframe = pd.DataFrame(audio_features)

    from sklearn import datasets
    from sklearn.preprocessing import StandardScaler

    user_cluster_table = user_audio_feature_dataframe[["danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness","liveness","valence","duration_ms"]]

    user_song_scaled = scaler.transform(user_cluster_table)
    user_input_normalized = pd.DataFrame(user_song_scaled)
    user_clusters = kmeans.predict(user_input_normalized)

    value_of_cluster_of_user = user_clusters[0]
    playlist_cluster_of_user = table_of_tables[table_of_tables["clusters"] == value_of_cluster_of_user]

    list_of_uri_of_cluster = playlist_cluster_of_user["uri"]
    a = list(list_of_uri_of_cluster)

    random_song = random.choice(a)

    random_song_track_info = spotify.track(random_song)

    random_song_track_info["name"]

Name of the song you like. If its in the top 100 I'll recommend you a new one?: clock
Song not in top 100
Who is the artist: coldplay


'在水中央'

---

---

---

---

---

---

---

---

---

---

# Other formulas

In [ ]:
kmeans.inertia_

K = range(2, 20)

# For each model, store the inertia in a list
inertia = []

for k in K:
    kmeans = KMeans(n_clusters=k,
        random_state=1234)
    kmeans.fit(table_of_tables_scaled)
    inertia.append(kmeans.inertia_)

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

plt.figure(figsize=(16,8))
plt.plot(K, inertia, 'bx-')
plt.xlabel('k')
plt.ylabel('inertia')
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.title('Elbow Method showing the optimal k')


from sklearn.metrics import silhouette_score

K = range(2, 20)
silhouette = []

for i in K:
    kmeans = KMeans(n_clusters=i, random_state=1234)
    prediction = kmeans.fit_predict(table_of_tables_scaled)    
    score = silhouette_score(table_of_tables_scaled, prediction)
    silhouette.append(score)

plt.figure(figsize=(16,8))
plt.plot(K, silhouette, 'bx-')
plt.xlabel('k')
plt.ylabel('silhouette score')
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.title('Elbow Method showing the optimal k')

In [ ]:
columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature']

TABLE = pd.DataFrame(columns = columns)



chinese_top_hits_full_playlist = []
playlist_link = "https://open.spotify.com/playlist/3CgsCde2Ol4MYOHqVdU56S"

for i in range(0,1000,100):
    chinese_top_hits_full_playlist.append(spotify.user_playlist_tracks(user="spotify",playlist_id=playlist_link,offset=i))

uri_full_playlist = []

for songs in chinese_top_hits_full_playlist:
    for elem in songs["items"]:
        uri_full_playlist.append(elem["track"]["uri"])

clean_uris = []

for uri in uri_full_playlist:
    if len(uri) == 36: 
        clean_uris.append(uri)

audio_features_playlist = []

for uri in clean_uris:
    audio_features_playlist.append(spotify.audio_features(uri))
    

full_table_playlist = pd.concat([pd.DataFrame(d) for d in audio_features_playlist])

TABLE = table_of_tables.append(full_table_playlist)

TABLE.drop_duplicates()